In [ ]:
def install(name):
    subprocess.call(['pip', 'install', name])
import subprocess
import numpy as np
from numpy import dot
from numpy.linalg import norm
import math
import os
import csv
import os
import pandas as pd
import matplotlib.pyplot as plt
import mplcursors
import tkinter as tk
from tkinter import filedialog
import ipywidgets as widgets
from IPython.display import display
import textstat
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, cross_val_predict, GroupKFold
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler, LabelEncoder
# from imblearn.over_sampling import SMOTE
import warnings
warnings.filterwarnings("ignore")
import numpy as np
%matplotlib notebook
from bnlp import POS,NER
from bnlp.corpus import stopwords
from bnlp import NLTKTokenizer
try:
	from bangla_stemmer.stemmer import stemmer
except:
	install('bangla-stemmer')
	from bangla_stemmer.stemmer import stemmer
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize, sent_tokenize
import nltk
from collections import Counter
import numpy as np
from numpy.linalg import norm
from sentence_transformers import SentenceTransformer, util
from pathlib import Path


try:
	from pydrive.auth import GoogleAuth
	from pydrive.drive import GoogleDrive
except:
	install('pydrive')
	from pydrive.auth import GoogleAuth
	from pydrive.drive import GoogleDrive

def LABSE_Embedding(c, Lang):
	if Lang == 'Bn':
		bnltk=NLTKTokenizer()
		sentences=bnltk.sentence_tokenize(c)
	else:
		sentences=nltk.sent_tokenize(c)
	model = SentenceTransformer('sentence-transformers/LaBSE')
	embeddings = model.encode(sentences, convert_to_tensor=True)
	return embeddings,sentences

def LABSE_SemanticSimilarity(c1,c2, Lang):
    B_Embedding1,sentences1=LABSE_Embedding(c1, Lang)
    B_Embedding2,sentences2=LABSE_Embedding(c2, Lang)
    #Compute cosine-similarities
    cosine_scores = util.cos_sim(B_Embedding1, B_Embedding2)
    values, indices=cosine_scores.max(axis=1)
    #for i in range(len(sentences1)):
    #    print("{} \t\t {} \t\t Score: {:.4f}".format(sentences1[i], sentences2[indices[i]], values[i]))
    
    mean_sim=values.mean().item()
    #print("Average of Maximum Similarity:", mean_sim)
    return mean_sim


def readability(c, Lang="En"):
    F=[]
    F.append(textstat.flesch_reading_ease(c))
    F.append(textstat.flesch_kincaid_grade(c))
    F.append(textstat.smog_index(c))
    F.append(textstat.coleman_liau_index(c))
    F.append(textstat.automated_readability_index(c))
    F.append(textstat.dale_chall_readability_score(c))
    F.append(textstat.difficult_words(c))
    F.append(textstat.linsear_write_formula(c))
    F.append(textstat.gunning_fog(c))
    #F.append(textstat.text_standard(c))
    F.append(textstat.fernandez_huerta(c))
    F.append(textstat.szigriszt_pazos(c))
    F.append(textstat.gutierrez_polini(c))
    F.append(textstat.crawford(c))
    F.append(textstat.gulpease_index(c))
    ####F.append(textstat.osman(c)) ------------------> Bk
    return F
    

def read_file(flname):
	fp=open(flname,'rb')
	c=fp.read()
	n=c.decode()
	fp.close()
	return n
def Read_config():
    fp=open('C:\\Users\\RITUJA\\Desktop\\AuthorshipAttributionFeatures\\config.txt')
    n=fp.read()
    f=n.replace('\n',' ')
    return eval (f)

def count_words(c, Lang="En"):
	if Lang == 'Bn':
		bnltk=NLTKTokenizer()
		L=len(bnltk.word_tokenize(c))
	else:
		L=len(nltk.word_tokenize(c))
	return L

def count_lines(c, Lang="En"):
	if Lang == 'Bn':
		bnltk=NLTKTokenizer()
		L=len(bnltk.sentence_tokenize(c))
	else:
		L=len(nltk.sent_tokenize(c))
	return L

def count_unique_words(c, Lang="En"):
	if Lang == 'Bn':
		bnltk=NLTKTokenizer()
		L=len(set(bnltk.word_tokenize(c)))
	else:
		L=len(set(nltk.word_tokenize(c)))
	return L

def POS_tag_Prob_Dist(c,Lang='En'):
	PST=[]
	if Lang=='Bn':
		bnltk=NLTKTokenizer()
		L1=len(bnltk.word_tokenize(c))
		bn_pos = POS()
		model_path = "C:\\Users\\RITUJA\\Desktop\\AuthorshipAttributionFeatures\\model\\bn_pos.pkl"
		L= bn_pos.tag(model_path, c)
		#print(L)
		the_count = Counter(tag for _, tag in L)
		TS=sum(the_count.values())
		BL= [ 'O','ALC','VAUX','NC','NP','NV','NST','PP','PPR','PRF','PRC','PRL','PWH','DAB','DRL','DWH','JJ','JQ','JINT','VM','VA','AMN','LPR','LPS','LFU','PPC','CSB','CIN','CCD','CCL','AGR','EMP','TOP','DLIM','HON','NEG','EXCL','TERM','DUB','SIM','INCL','COM','CX','NUMR','NUMS','NUMC','NUMO','RDP','RDF','RDS','UNK','PU']
		#print(BL)
		for t in BL:
			PST.append(the_count[t]/TS)
	else:
		L1=nltk.word_tokenize(c) #-----------> Bk
		L=nltk.pos_tag(L1)
		print(L)
		the_count = Counter(tag for _, tag in L)
		TS=sum(the_count.values())
		EL=['CC','CD','DT','EX','FW','IN','JJ','JJR','JJS','LS','MD','NN','NNS','NNP','NNPS','PDT','POS','PRP','PRP$','RB','RBR','RBS','RP','SYM','TO','UH','VB','VBD','VBG','VBN','VBP','VBZ','WDT','WP','WP$','WRB']
		for t in EL:
			PST.append(the_count[t]/TS)
	return PST
    
def cosine_similarity(L1,L2):
	cos_sim=dot(L1,L2)/(norm(L1)*norm(L2))
	return cos_sim

def NovelWords(c, Lang="En"):
	U=count_unique_words(c, Lang)
	T=count_words(c, Lang)
	return [U/T]

def Count_punctuations(c,Lang='En'):
	PUNC = []
	if Lang=='Bn':
		puncBL=['@', '!', ':', ';', ',', '#', '$', '%', '^', '(',')','।', '&', '*', '_', "'",'`', '-', '~', '{','}', '.', '?', '/','\\']
		for p in puncBL:
			PUNC.append(c.count(p)/count_words(c,Lang))
	else:
		puncEN=['!','"','#','$','%','&','(',')','*','+', '-','.','/',':',';','<','=','>','?','[',']','@','/','\\','^','_','`','{','}','~','|']
		for p in puncEN:
				PUNC.append(c.count(p)/count_words(c,Lang))
	return PUNC

def AvgSentenceLength(c, Lang="En"):
	if Lang == 'Bn':
		bnltk=NLTKTokenizer()
		Lns=bnltk.sentence_tokenize(c)
	else:
		Lns=nltk.sent_tokenize(c)
	num_of_sen=len(Lns)
        
	sen_len=0 #Number of words in a sentence
	for l in Lns:
		sen_len+=count_words(l,Lang)
	return [sen_len/num_of_sen]

def BOW(c1,c2):
	cv = CountVectorizer()
	X = np.array(cv.fit_transform([c1,c2]).todense())
	return X

def FeatureVector(c, Lang,Cfg):
	Fv=[]
	for i in Cfg:
		if i[1]:
			Fv+=i[0](c,Lang)
	return Fv
def NER_tag_Prob_Dist(c,Lang='En'):
	NERST=[]
	if Lang=='Bn':
		bnltk=NLTKTokenizer()
		L1=len(bnltk.word_tokenize(c))
		bn_ner = NER()
		model_path = "C:\\Users\\RITUJA\\Desktop\\AuthorshipAttributionFeatures\\model\\bn_pos.pkl"
		L= bn_ner.tag(model_path, c)
		#print(L)
		the_count = Counter(tag for _, tag in L)
		TS=sum(the_count.values())
		NERBL=['O','B-PER','I-PER','E-PER','S-PER','B-ORG','I-ORG','E-ORG','S-ORG','B-LOC','I-LOC','E-LOC','S-LOC']
		#print(NERBL)
		for t in NERBL:
			NERST.append(the_count[t]/TS)
	else:
		L1=nltk.word_tokenize(c)
		tagging=nltk.pos_tag(L1)
		L=nltk.ne_chunk(tagging) 
		L1=[(ne.leaves()[0][0],ne.label()) for ne in L if type(ne) == nltk.Tree]
		the_count = Counter(tag for _, tag in L1)      
		TS=sum(the_count.values())
		NEREL=['B-PER','I-PER','E-PER','S-PER','B-ORG','I-ORG','E-ORG','S-ORG','B-LOC','I-LOC','E-LOC','S-LOC']
		for t in NEREL:
			NERST.append(the_count[t]/TS)
	return NERST

def Stemming_Dist(c,Lang='En'):
    stmLs=[]
    if Lang=='Bn':
        bnltk=NLTKTokenizer()
        L=bnltk.word_tokenize(c)
        stmr = stemmer.BanglaStemmer()
        stmLs=[stmr.stem(word) for word in L]
    else:
        L=nltk.word_tokenize(c)
        ps=PorterStemmer()
        stmLs=[ps.stem(word) for word in L]
        
    return stmLs

def FindSimilarity(QueryPath, DocPath, Lang):
	Lst,X=[], [[0],[0]]
	c1= read_file(QueryPath)
	Cfg=Read_config()

	for (root,dirs,files) in os.walk(DocPath, topdown=True):
		for file in files:	
			DocPathFull=os.path.join(root,file)
			if Path(QueryPath) == Path(DocPathFull): 
				print("Not Comparing with same file ...")
				continue
			c2= read_file(DocPathFull)
			if Cfg[1]['BOW']:
				if Cfg[0]['Stemming_Dist']:
					X=BOW(" ".join(Stemming_Dist(c1))," ".join(Stemming_Dist(c2)))
				else:
					X=BOW(c1,c2)
			L1=FeatureVector(c1,Lang,Cfg[2:]) + list( X[0])
			L2=FeatureVector(c2, Lang,Cfg[2:]) + list( X[1])
			Syntactic_Similarity=cosine_similarity(L1,L2)
			if Cfg[1]['LABSE_SemanticSimilarity']:
							Semantic_Similarity=LABSE_SemanticSimilarity(c1,c2, Lang)
							if math.isnan(Syntactic_Similarity) : Similarity_Score=Semantic_Similarity
							else: Similarity_Score=(Syntactic_Similarity+Semantic_Similarity)/2
			else:
				Similarity_Score= Syntactic_Similarity
			Lst.append([Similarity_Score,DocPathFull])
			Lst.sort(reverse=True)
	Lst=np.array(Lst, dtype=object)
	return Lst[:,0].mean(), Lst



def Evaluations(MyPath,DirsLs,flname,Lang = 'Bn'):
    import csv
    rows=[]
    cnt=0
    T=0
    fp=open('C:\\Users\\RITUJA\\Desktop\\AuthorshipAttributionFeatures\\config.txt')
    n=fp.read()
    conf=Read_config()
    fp.close()
	
    for d in DirsLs:  
           for (root,dirs,files) in os.walk(os.path.join(MyPath,d), topdown=True):
                for file in files:
                    QueryPathFull=os.path.join(root,file)
                    SimScore=[]
                    TmpResult=[]
                    for d1 in DirsLs:
                        DocPath=os.path.join(MyPath,d1)
                        #print(QueryPathFull, "<-->", DocPath+"\\")
                        AvgSim,Vals=FindSimilarity(QueryPathFull,DocPath,Lang)
                        #AvgSim=random.random()
                        SimScore+=[AvgSim]
                    #print(d,SimScore)
                    Label=False
                    if d == DirsLs[SimScore.index(max(SimScore))]: 
                        Label=True
                        T+=1
                    cnt+=1
                        
                    TmpResult=[QueryPathFull,conf]+SimScore+[Label]
                    rows.append(TmpResult)
    fields = ['FileName', 'Configuration']+DirsLs+['Prediction'] 
    #print(fields)
    with open(str(T/cnt)+"_"+flname, 'w',newline='') as f:
        # using csv.writer method from CSV package
        write = csv.writer(f)      
        write.writerow(fields)
        write.writerows(rows)        
    print("Accuracy: ", T/cnt)
    return rows



def Evaluations(MyPath,DirsLs,flname,Lang = 'Bn'):
    import csv
    rows=[]
    cnt=1
    T=0
    fp=open('C:\\Users\\RITUJA\\Desktop\\AuthorshipAttributionFeatures\\config.txt')
    n=fp.read()
    conf=Read_config()
    fp.close()
	
    for d in DirsLs:
           for (root,dirs,files) in os.walk(os.path.join(MyPath,d), topdown=True):
                for file in files:
                    QueryPathFull=os.path.join(root,file)
                    SimScore=[]
                    TmpResult=[]
                    for d1 in DirsLs:
                        DocPath=os.path.join(MyPath,d1)
                        #print(QueryPathFull, "<-->", DocPath+"\\")
                        AvgSim,Vals=FindSimilarity(QueryPathFull,DocPath,Lang)
                        #AvgSim=random.random()
                        SimScore+=[AvgSim]
                    #print(d,SimScore)
                    Label=False
                    if d == DirsLs[SimScore.index(max(SimScore))]: 
                        Label=True
                        T+=1
                    cnt+=1
                        
                    TmpResult=[QueryPathFull,conf]+SimScore+[Label]
                    rows.append(TmpResult)
    fields = ['FileName', 'Configuration']+DirsLs+['Prediction'] 
    #print(fields)
    with open(flname, 'w',newline='') as f:
        # using csv.writer method from CSV package
        write = csv.writer(f)      
        write.writerow(fields)
        write.writerows(rows)        
    print("Accuracy: ", T/cnt)
    return rows


def EvaluateModel():
    import os
    import random
    import csv
    MyPath= "C:\\Users\\RITUJA\\Desktop\\NLTR"
    DirsLs=[
	"Aparupa Dasgupta",
	"Safia Pailan",    
    ]
    flname= 'D://Evaluation.csv'
    Result=Evaluations(MyPath,DirsLs,flname)
    #print(Result)


def CreateCSV():
	Cfg=Read_config()
	Ls=[]
	Lang="Bn"
	DocPath="C:\\Users\\RITUJA\\NLTR\\"
	Fields = []

	BL= [ 'O','ALC','VAUX','NC','NP','NV','NST','PP','PPR','PRF','PRC','PRL','PWH','DAB','DRL','DWH','JJ','JQ','JINT','VM','VA','AMN','LPR','LPS','LFU','PPC','CSB','CIN','CCD','CCL','AGR','EMP','TOP','DLIM','HON','NEG','EXCL','TERM','DUB','SIM','INCL','COM','CX','NUMR','NUMS','NUMC','NUMO','RDP','RDF','RDS','UNK','PU']

	puncBL=['@', '!', ':', ';', ',', '#', '$', '%', '^', '(',')','|', '&', '*', '_', "'",'`', '-', '~', '{','}', '.', '?', '/','\\']

	NERBL=['O','B-PER','I-PER','E-PER','S-PER','B-ORG','I-ORG','E-ORG','S-ORG','B-LOC','I-LOC','E-LOC','S-LOC']
    
	RL=['flesch_reading_ease','flesch_kincaid_grade','smog_index','coleman_liau_index','automated_readability_index','dale_chall_readability_score','difficult_words','linsear_write_formula','gunning_fog','fernandez_huerta','szigriszt_pazos','gutierrez_polini','crawford','gulpease_index']


	for Config in Cfg:
		if isinstance(Config, dict):
			for key, value in Config.items():
				if value == 1:
					Fields.append(key)
		elif isinstance(Config, tuple):
			field, value = Config
			if value == 1:
				if field.__name__ == 'NER_tag_Prob_Dist':
					Fields += NERBL
				elif field.__name__ == 'readability':
					Fields += RL
				elif field.__name__ == 'POS_tag_Prob_Dist':
					Fields += BL 
				elif field.__name__ == 'Count_punctuations':
					Fields += puncBL
				else:Fields.append(field.__name__)

                
	Fields += ['Author', 'Filename'] 
	print(Fields)                
	for root,dr,files in os.walk(DocPath):
		for f in files:
			flnm=os.path.join(root,f)
			#print(flnm)
			drnm=flnm.split("\\")
			drnm=drnm[drnm.index("NLTR")+1]
			#print(drnm)
			c=read_file(flnm)
			FV=FeatureVector(c, Lang,Cfg[2:])
			ls=FV+[drnm]+["\\".join(flnm.split("\\")[-2:])]
# 			print(ls)
			Ls.append(ls)
	with open("D:\\sample_result.csv","w") as f:
		writer_object=csv.writer(f)
# 		Fields=[i.encode("utf-8") for i in Fields]
		writer_object.writerow(Fields)
		writer_object.writerows(Ls)
	print(Ls)
	print(writer_object)
    

def VisualizeScatter(flname,fx,fy):
    import pandas as pd 
    import numpy as np
    import matplotlib.pyplot as plt
    import seaborn as sns
    import warnings
    warnings.filterwarnings("ignore")
    import mplcursors
    np.random.seed(42)
    df=pd.read_csv(flname)

    sns.set(style="darkgrid")

    sns.relplot(x=fx, y=fy, hue="Author", data=df);
    plt.savefig("D://FeatureComp.png")
    fig, ax = plt.subplots()
    author_colors = {author: i for i, author in enumerate(df['Author'].unique())}
    colors = df['Author'].map(author_colors)

    ax.scatter(df[fx], df[fy], c=colors)
    ax.set_title("Hover effect")
    crs = mplcursors.cursor(ax, hover=True)
    def format_annotation(sel):
        index = sel.target.index
        point = df.iloc[index]
        text = 'Author: {}\nfx: {}\nfy: {}'.format(
            point['Author'], point[fx], point[fy])
        sel.annotation.set_text(text)

    crs.connect("add", format_annotation)
    plt.show()

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

def generate_prediction(model_type):
    existing_data = pd.read_csv('D:\\sample_result.csv', skip_blank_lines=True).dropna()

    features = ['NovelWords', 'AvgSentenceLength', 'flesch_reading_ease', 'flesch_kincaid_grade',
                'smog_index', 'coleman_liau_index', 'automated_readability_index', 'dale_chall_readability_score',
                'difficult_words', 'linsear_write_formula', 'gunning_fog', 'fernandez_huerta',
                'szigriszt_pazos', 'gutierrez_polini', 'crawford', 'gulpease_index']

    X = existing_data[features]
    y = existing_data['Author']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=None)

    if model_type == 'Logistic Regression':
        clf = LogisticRegression(max_iter=1000)
    elif model_type == 'Random Forest Classifier':
        clf = RandomForestClassifier(n_estimators=100, random_state=None)
    else:
        print("Invalid model type.")
        return None
    
    clf.fit(X_train, y_train)

    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=None)
    cv_scores = cross_val_score(clf, X, y, cv=cv, scoring='accuracy')
    accuracy = cv_scores.mean() * 100

    df = pd.read_csv('D:\\samp_result.csv', skip_blank_lines=True).dropna()
    X_test = df[features]
    y_test = df['Author']
    pred = clf.predict(X_test)
    pred_proba = clf.predict_proba(X_test)

    label_encoder = LabelEncoder()
    label_encoder.fit(y_train)

    pred_label_indices = label_encoder.transform(pred)
    pred_author_names = label_encoder.inverse_transform(pred_label_indices)
#     print("Authors Names:",y.unique())  
#     print("Scores:", cv_scores)
    cvas = {}
    for author, score in zip(y.unique(), cv_scores):
        cvas[author] = score

    print("Cross Validation Accuracy Scores:", cvas)

    print("Accuracy: %.2f%%" % accuracy)
    train_pred = clf.predict(X_train)
    train_report = classification_report(y_train, train_pred)
    print("Classification Report (Training Dataset):\n", train_report)
    print("Prediction:", pred_author_names)

    probabilities = {}
    for author, probs in zip(clf.classes_, pred_proba[0]):
        probabilities[author] = f"{probs:.4f}"

    print("Probabilities:", probabilities)

    return cvas, accuracy, train_report, pred_author_names, pred_proba

def generate_plot(text, fx, fy, model_type):
    Fields = ['NovelWords', 'AvgSentenceLength', 'flesch_reading_ease', 'flesch_kincaid_grade', 'smog_index', 'coleman_liau_index', 'automated_readability_index', 'dale_chall_readability_score', 'difficult_words', 'linsear_write_formula', 'gunning_fog', 'fernandez_huerta', 'szigriszt_pazos', 'gutierrez_polini', 'crawford', 'gulpease_index', 'Author']
    Lang = "Bn"
    c = text
    Cfg = Read_config()
    FV = FeatureVector(c, Lang, Cfg[2:])
    ls = FV + ["Query"]
    with open("D:\\samp_result.csv", "w", newline='') as f:
        writer_object = csv.writer(f)
        writer_object.writerow(Fields)
        writer_object.writerow(ls)

    if fx and fy:
        existing_data = pd.read_csv('D:\\sample_result.csv', skip_blank_lines=True).dropna()
        df = pd.read_csv('D:\\samp_result.csv', skip_blank_lines=True).dropna()
        new_x = df[fx]
        new_y = df[fy]
        combined_x = pd.concat([existing_data[fx], new_x])
        combined_y = pd.concat([existing_data[fy], new_y])
        #print(existing_data['Author'])
        combined_colors = pd.concat([existing_data['Author'], df['Author']])
        author_colors = {author: i for i, author in enumerate(combined_colors.unique())}
        print(author_colors.keys())
        
        legend_labels = list(author_colors.keys())
        legend_handles = []
        for i,label in enumerate(legend_labels):
            color = plt.cm.tab10(author_colors[label]) 
            legend_handles.append(plt.Line2D([0], [0], marker='o', color=color, label=label, markersize=8))

        author_colors1 = {author: plt.cm.tab10(author_colors[author]) for author in combined_colors.unique()}
        colors = combined_colors.map(author_colors1)

        fig, ax = plt.subplots()
        scatter = ax.scatter(combined_x, combined_y, c=colors) 
        ax.set_title("Combined Plot")
        ax.set_xlabel(fx)
        ax.set_ylabel(fy)

        crs = mplcursors.cursor(scatter)

        def format_annotation(sel):
            index = sel.target.index
            point = pd.concat([existing_data, df]).iloc[index]
            text = 'Author: {}\n{}: {}\n{}: {}'.format(point['Author'], fx, point[fx], fy, point[fy])
            sel.annotation.set_text(text)

        crs.connect("add", format_annotation)
        

        ax.legend(handles=legend_handles, loc='best')

        #ax.legend()
        plt.savefig('D://image.png')
        plt.show()
    else:
        print("Please provide values for fx and fy.")

def CSVandCombinedPlot(model_type):
    import tkinter as tk
    from tkinter import Tk,filedialog
    import ipywidgets as widgets
    from IPython.display import display
    
    def browse_file():
        root = Tk()
        root.withdraw()
        file_path = filedialog.askopenfilename()

        with open(file_path, 'r', encoding='utf-8') as file:
            text = file.read()

        text_area.value = text
    
    fields = ['NovelWords', 'AvgSentenceLength', 'flesch_reading_ease', 'flesch_kincaid_grade', 'smog_index', 'coleman_liau_index', 'automated_readability_index', 'dale_chall_readability_score', 'difficult_words', 'linsear_write_formula', 'gunning_fog', 'fernandez_huerta', 'szigriszt_pazos', 'gutierrez_polini', 'crawford', 'gulpease_index', 'Author']
    prediction_button = widgets.Button(description="Predict")

    def generate_predictions(button):
        cvas,accuracy,train_report,pred_author_names,proba_dict = generate_prediction(model_type)

    prediction_button.on_click(generate_predictions)

    text_area = widgets.Textarea(value='', placeholder='Enter text here', rows=4)
    browse_button = widgets.Button(description="Browse File")
    browse_button.on_click(lambda button: browse_file())

    fx_dropdown = widgets.Dropdown(options=fields, description="fx:")
    fy_dropdown = widgets.Dropdown(options=fields, description="fy:")
    button = widgets.Button(description="Generate Plot")

    button.on_click(lambda button: generate_plot(text_area.value, fx_dropdown.value, fy_dropdown.value, model_type))

    display(prediction_button, text_area, browse_button, fx_dropdown, fy_dropdown, button)

if __name__ == '__main__':
    import os
    from ipywidgets import interactive, Button, Textarea
    import ipywidgets as widgets
    from IPython.display import display
    
    model_type_dropdown = widgets.Dropdown(options=['Logistic Regression', 'Random Forest Classifier'], description="Model Type:")
    display(model_type_dropdown)
    
    def model_type_selected(change):
        model_type = change.new
        CSVandCombinedPlot(model_type)

    model_type_dropdown.observe(model_type_selected, 'value')
